# Стратегия входа и выхода по SMA (200)  

* 200-дневная средняя SМА(200) разделяет бычий и медвежий рынок.
* Если цена актива над SМА(200), то он в бычьем тренде, если под - в медвежьем.
* Стратегия входа и выхода по SМА(200) предполагает, что мы открываем позицию при закреплении цены над МА(200) в течение n дней и закрываем позицию при закреплении цены над МА(200) в течение n дней.
* Приведенный ниже код позволяет протестировать данную стратегию на quantopian.com.

In [ ]:
# Импортирует библиотеку talib для построения скользящей средней. 
import talib


def initialize(context):
    """
    Создаем функцию initialize(context) для начала работы.
    Указываем нужный тикер актива через свойство context.security.
    Создаем функцию schedule_function(), в которой проверяем актив каждый день, через 1 час после открытия рынка.
    Указываем нужное число дней нахождения цены актива относительно SMA(200).
    """
    context.security = symbol('SPY')
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open(hours=1))
    context.days = 5

    
def rebalance(context, data):
    """
    Создаем функцию rebalance(context, data).
    Здесь проверяются сигналы.
    """
    # Получения истории цен закрытия актива за последние 500 дней.
    hist = data.history(context.security, ['high', 'low', 'close'], 500, '1d')
    
    # Строим на основе talib простую скользящую SMA по ценам закрытия за 200 дней.
    # Создаем переменную spread, чтобы узнать положение цены относительно SMA(200) days дней назад.    
    ma_long = talib.SMA(hist['close'], timeperiod=200)
    spread = (hist['close'] - ma_long)[-context.days:]
     
    # Задаем условия для открытия или закрытия позиции.
    # Если цена находится над SMA(200) days дней, открываем позицию.  
    # Если цена находится под SMA(200) days дней, закрываем позицию.
    # Выводим соответствующие уведомления в quantopian.com с помощью метода log.info().    
    if len(spread[spread > 0]) == context.days:
        order_target_percent(context.security, 1.)
        log.info('Buying shares')

    elif len(spread[spread < 0]) == context.days:
        order_target_percent(context.security, 0.)
        log.info('Selling shares')
        
    # Подписываем график тестирования на quantopian.com.        
    record(SMA=ma_long[-1], price=hist['close'][-1])


# Код стратегии без комментариев

In [ ]:
import talib


def initialize(context):
    context.security = symbol('QQQ')
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open(hours=1))
    context.days = 5
  

def rebalance(context, data):
    hist = data.history(context.security, ['close'], 500, '1d')
    ma_long = talib.SMA(hist['close'], timeperiod=200)
    
    spread = (hist['close'] - ma_long)[-context.days:]
    
    if len(spread[spread > 0]) == context.days:
        order_target_percent(context.security, 1.)
        log.info('Buying shares')

    elif len(spread[spread < 0]) == context.days:
        order_target_percent(context.security, 0.)
        log.info('Selling shares')
        
    record(SMA=ma_long[-1], price=hist['close'][-1])

# Стратегия входа и выхода по пересечению SMA (200) и SMA (50)

* Пересечение 50-дневной средней SМА(50) относительно 200-дневной SМА(200) является сигналом смены тенденции.
* Если SМА(50) пересекает SМА(200) снизу вверх, то тренд с медвежьего сменился на бычий.
* Если SМА(50) пересекает SМА(200) сверху вниз, то тренд с бычьего сменилмя на медвежий.
* Стратегия входа и выхода по пересечению SMA (200) и MA (50) предполагает, 
* что мы открываем позицию при входе актива в бычий тренд и закрываем - при входе смене тендеции на медвежью. 
* Приведенный ниже код позволяет протестировать данную стратегию на quantopian.com.

In [ ]:
# Импортирует библиотеку talib для построения скользящей средней. 
import talib


def initialize(context):
    """
    Создаем функцию initialize(context) для начала работы.
    Указываем нужный тикер актива через свойство context.security.
    Создаем функцию schedule_function(), в которой проверяем актив каждый день, 
    через 1 час после открытия рынка.
    """
    context.security = symbol('SPY')
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open(hours=1))

    
def rebalance(context, data):
    """
    Создаем функцию rebalance(context, data) для получения истории цен закрытия 
    актива за последние 500 дней.
    Строим с помощью talib простые скользящие средние SMA по ценам закрытия 
    за 50 и 200 дней и получаем их последние значения с помощью индекса [-1].
    """
    hist = data.history(context.security, ['close'], 500, '1d')
    
    ma_short = talib.SMA(hist['close'], timeperiod=50)[-1]  
    ma_long = talib.SMA(hist['close'], timeperiod=200)[-1]

    # Задаем условия для открытия или закрытия позиции.
    # Если SMA(500) находится над SMA(200), открываем позицию.  
    # Если SMA(500) находится под SMA(200), закрываем позицию.
    # Выводим соответствующие уведомления в quantopian.com с помощью метода log.info().
    if (ma_short > ma_long):
        order_target_percent(context.security, 1.)
        log.info('Buying shares')

    elif (ma_short < ma_long):
        order_target_percent(context.security, 0.)
        log.info('Selling shares')
        
    # Подписываем график тестирования на quantopian.com.         
    record(MA1=ma_short, MA2=ma_long, price=hist['close'][-1])

# Код стратегии без комментариев

In [ ]:
import talib


def initialize(context):
    context.security = symbol('SPY')
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open(hours=1))

    
def rebalance(context, data):
    hist = data.history(context.security, ['close'], 500, '1d')
    
    ma_short = talib.SMA(hist['close'], timeperiod=50)[-1]  
    ma_long = talib.SMA(hist['close'], timeperiod=200)[-1]

    if (ma_short > ma_long):
        order_target_percent(context.security, 1.)
        log.info('Buying shares')

    elif (ma_short < ma_long):
        order_target_percent(context.security, 0.)
        log.info('Selling shares')
        
    record(MA1=ma_short, MA2=ma_long, price=hist['close'][-1])

# Стратегия "Купи и держи"

In [ ]:
import talib


def initialize(context):
    context.security = symbol('SPY')
    
    schedule_function(rebalance_buy, date_rules.every_day(), time_rules.market_open(hours=1))

    
def rebalance_buy(context, data):
    order_target_percent(context.security, 1.) 